In [1]:
import srsly
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_colwidth = 100

## pyttsx3
- cannot synthesize a list of strings sequentially? - issue replicated with script, not a notebook issue
- cannot supply "complex" output dir - eg. nested within otso-rand bucket
- can only process list of two strings sequentially
- outputed audio files do not play? when generated from script or in sequence
- tested a variety of different transcript outputs 

In [210]:
self_chat = list(srsly.read_jsonl('/Users/samhardyhey/Downloads/TransformerGenerator_1_TransformerGenerator_2_selfchat.jsonl'))

In [249]:
def format_episode(episode_raw):
    # as a dataframe
    episode = []
    for e in episode_raw['dialog']:
        episode.extend([{'id': ee['id'], 'text': ee['text']} for ee in e])

    return (pd.DataFrame(episode)
              .reset_index()
              .rename(mapper={'index': 'exchange_index'}, axis='columns')
            )

In [21]:
transcript = """[TransformerGenerator_1]: hi i am from the city how are you ? i live in california . where are you from ?
   [TransformerGenerator_2]: i ' d love to visit cali . i ' ve never been there . i am 56 .
[TransformerGenerator_1]: i love california its beautiful ! i ' m in my mid 50 ' s . where do you live ?
   [TransformerGenerator_2]: i live on the west coast . i used too be pretty , but i am not anymore . you ?
[TransformerGenerator_1]: i live near san francisco . what do you do for fun ? i am an alcoholic but i love it !
   [TransformerGenerator_2]: i love to fantasize . i have dreamed of being the world and taking over . do you ?
[TransformerGenerator_1]: i do not ! i love the outdoors . i love being in the mountains ! what are you up to ?
   [TransformerGenerator_2]: i was a painter , now i work as a housemaid . you ever been to cali ?
[TransformerGenerator_1]: i have not , but my family would love to go . i grew on a very small farm .
   [TransformerGenerator_2]: that ' s nice . where did you grow the food ? do you have any hobbies ?"""

processed_transcripts = []
for e in transcript.split('\n'):
    # remove brackets, and computer generated names
    ee = e.strip().replace('[', '').replace(']', '').replace(
        'TransformerGenerator', 'Speaker').strip()
    processed_transcripts.append(ee)

In [ ]:
# potentially split each utterance into sentences, allow for padding/spacing in audio?
import spacy
nlp = spacy.load('en_core_web_md')

In [ ]:
engine.say(list(nlp(processed_transcripts[1].split(':')[1]).sents)[1])
engine.runAndWait()

In [2]:
import pyttsx3
engine = pyttsx3.init()

In [3]:
# potentially choose a random persona for each chat transcript?
voices = engine.getProperty('voices')
english_voices = [e for e in voices if any(
    'en' in lang for lang in e.languages)]

# for voice in english_voices:
#     print("\nVoice:")
#     print("ID: %s" % voice.id)
#     print("Name: %s" % voice.name)
#     print("Age: %s" % voice.age)
#     print("Gender: %s" % voice.gender)
#     print("Languages Known: %s" % voice.languages)

In [8]:
engine.setProperty('rate', 130) # slow it down
engine.setProperty('voice', english_voices[5].id) # choose a voice

# engine.say("i love california its beautiful ! i ' m in my mid 50 ' s . where do you live ?")
# engine.runAndWait()

engine.save_to_file("i love california its beautiful ! i ' m in my mid 50 ' s . where do you live ?", 'test.mp3')
engine.runAndWait()

In [ ]:
from pathlib import Path
output_dir = Path(
    '/Users/samhardyhey/Desktop/otso-rand/experiment_artefacts/stt_data_synthesis/exp_b/doodle_call_a')
output_dir.mkdir(
    exist_ok=True, parents=True) if output_dir.exists() == False else None

In [ ]:
for idx, e in enumerate(processed_transcripts):
    # format text/save file
    text = e.split(': ')[1].lower()
#     save_path = output_dir / f"{idx}_{e.split(': ')[0].lower()}.mp3"
    print(f"{idx}_{e.split(': ')[0].lower()}.mp3")
    
    # synthesize, save locally? cannot save to "complex" dir?
    f"{idx}_{e.split(': ')[0].lower()}.mp3"
#     engine.save_to_file(text, f"{idx}_{e.split(': ')[0].lower()}.mp3")
#     engine.runAndWait()

## Using google speech to sound
- https://github.com/pndurette/gTTS
- Risk of IP banning? limits unclear https://github.com/pndurette/gTTS/discussions/293
- Customizable text pre-processors which can, for example, provide pronunciation corrections
- Better audio form, WRT pauses between sentences etc.
- Outputs play outside of main application!

In [26]:
# basic use
import gtts
from playsound import playsound

text = 'hi i am from the city how are you ? i live in california . where are you from '
tts = gtts.gTTS(text, lang='en', tld='com.au', slow=True)
tts.save('./hello.mp3')
playsound("hello.mp3")

In [23]:
from pathlib import Path
import time
output_dir = Path(
    '/Users/samhardyhey/Desktop/otso-rand/experiment_artefacts/stt_data_synthesis/exp_b/doodle_call_a')
output_dir.mkdir(
    exist_ok=True, parents=True) if output_dir.exists() == False else None

In [187]:
# in place of pyttsx3
for idx, e in enumerate(processed_transcripts):
    time.sleep(1) # prevent IP banning?
    
    # format text/save file
    text = e.split(': ')[1].lower()
    speaker = int(e.split('_')[1][0])
    save_path = output_dir / f"{idx}_{e.split(': ')[0].lower()}.mp3"
    if speaker == 1:
        tts = gtts.gTTS(text, lang='en', tld='com.au', slow=True)
    elif speaker == 2:
        tts = gtts.gTTS(text, lang='en', tld='co.uk', slow=True)
    tts.save(save_path)

In [ ]:
## How to reconstruct a sequence across multiple channels?

In [29]:
import librosa
import soundfile

In [64]:
# samples = left_channel.get_array_of_samples()
# y, s = librosa.load('./hello.mp3') # new downsample rate
# soundfile.write(output_dir / f"{output_dir.name}_{Path(e).name}", y, s)

In [103]:
# import numpy as np

# pad = np.zeros(y.shape[0], dtype=np.float32)
# padded_audio = np.concatenate((pad, y, pad))
# soundfile.write('./hello_padded.wav', padded_audio, s)

In [ ]:
import pandas as pd

# collate utterance audio files into dataframe
audio_file_records = []
for e in list(output_dir.rglob('./*.mp3')):
    y, s = librosa.load(e.as_posix())  # new downsample rate
    audio_file_records.append(
        {'file': e.name, 'sample_array': y, 'sample_array_shape': y.shape[0]})

df = (pd.DataFrame(audio_file_records)
      .sort_values('file')
      .assign(channel=lambda x: x.file.apply(lambda y: int(y.split('_')[-1][0])))
      .reset_index(drop=True)
      )

In [190]:
# pad channel 1/2 chunks to ensure for interleaving pattern
channel_1_segments = []
channel_2_segments = []
for idx, e in df.iterrows():
    if idx % 2 == 0:
        print(f"{e.file} is even")
        # even indices are channel 1; starting first, indexing from zero
        channel_1_segments.append(e.sample_array) # the actual samples
        # pad alternating channel (channel 2) with equivalent size zero array
        channel_2_segments.append(np.zeros(e.sample_array.shape[0], dtype=np.float32))
    else:
        print(f"{e.file} is odd")
        # odd indices are channel 2
        channel_2_segments.append(e.sample_array)
        # otherwise, channel 2 length zero array
        channel_1_segments.append(np.zeros(e.sample_array.shape[0], dtype=np.float32))

0_speaker_1.mp3 is even
1_speaker_2.mp3 is odd
2_speaker_1.mp3 is even
3_speaker_2.mp3 is odd
4_speaker_1.mp3 is even
5_speaker_2.mp3 is odd
6_speaker_1.mp3 is even
7_speaker_2.mp3 is odd
8_speaker_1.mp3 is even
9_speaker_2.mp3 is odd


In [191]:
# temp save for channel 1/2 audio - saves as mono
default_sr = 22050
channel_1_padded = np.concatenate(channel_1_segments)
soundfile.write(output_dir / 'channel_1_temp.wav', channel_1_padded, default_sr)

channel_2_padded = np.concatenate(channel_2_segments)
soundfile.write(output_dir / 'channel_2_temp.wav', channel_2_padded, default_sr)

In [197]:
from pydub import AudioSegment

# consolidate into an interleaving, channel seperated source
left_channel = AudioSegment.from_wav(output_dir / 'channel_1_temp.wav')
right_channel = AudioSegment.from_wav(output_dir / 'channel_2_temp.wav')

# load individual channels...
stereo_sound = AudioSegment.from_mono_audiosegments(left_channel, right_channel)
stereo_sound.export(output_dir / 'consolidated_final.wav')

<_io.BufferedRandom name='/Users/samhardyhey/Desktop/otso-rand/experiment_artefacts/stt_data_synthesis/exp_b/doodle_call_a/consolidated_final.wav'>